# Tutorial 6: Tensorflow 

## Topics covered
1. Tensorflow - Installation and Basics
3. Linear Regression
4. Building a MLP

## Tensorflow

Taken from: https://www.tensorflow.org/

TensorFlow™ is an open source software library for numerical computation using data flow graphs. Nodes in the graph represent mathematical operations, while the graph edges represent the multidimensional data arrays (tensors) communicated between them. The flexible architecture allows you to deploy computation to one or more CPUs or GPUs in a desktop, server, or mobile device with a single API. TensorFlow was originally developed by researchers and engineers working on the Google Brain Team within Google's Machine Intelligence research organization for the purposes of conducting machine learning and deep neural networks research, but the system is general enough to be applicable in a wide variety of other domains as well.

## Interesting Links and Tutorials
[Curated list of awesome Tensorflow experiments](https://github.com/jtoy/awesome-tensorflow)

[Video series on Tensorflow tutorials](https://www.youtube.com/watch?v=oYbVFhK_olY&list=PLSPWNkAMSvv5DKeSVDbEbUKSsK4Z-GgiP)

## Basics

**Installation**

https://www.tensorflow.org/install/

**Check available local devices**

**Variables and Placeholders**

**Namespace: Variable and Name scopes**

**Session**
1. Running computation graphs.
2. Get partition graphs allocations.

**Example: Matrix power $M^{1000}$**

**Sample example: Multiple GPUs**

## Linear Regression

Fit a equation of line to the given data.

$$ Y = Mx + C$$

Variables|Placeholders
--- | ---
$M$ and $C$ | $Y$ and $X$

**STEP 1**: Get the data

**STEP 2**: Initialize tensorflow variables, loss function and gradient descend operations

**STEP 3**: Perform the gradient descend

Plot the loss per iterations

**STEP 4:** Final loss and plot the fitted line

** More Exercise:** Can you fit $n$ degree polynomial? Write a generic function that takes value of $n$ and returns the computation model for:

$$Y = a_1 X^n + a_2 X^{n-1} ... + a_n$$

Rest everything should stay almost same. Can you tell why do we need `Placeholder` after solving the exercise?

## Multilayer Perceptron
**MLP Model**

1. Building two hidden layered neural network model. 
2. Each layer has Weights and Bias (variables to adjust). 
3. Activation of first layer is Sigmoid and last layer is Softmax. 
4. Loss function is Cross Entropy.

**Unit**|**Output**
-|-
Input | $I$
Layer 1| $L_1 = sigmoid(I \times W_1 + B_1)$
Layer 2| $L_2 = Sigmoid(L_1 \times W_2 + B_2)$
Output | $out = L_2*W_{out} + B_{out}$
Loss | $loss(out, actual)$

**Tensorflow Variables**

|Layer|Variables|Size|
|-|-|
|Layer 1|$W_1$|$D \times N_1$|
|Layer 1|$B_1$|$D \times 1$|
|Layer 2|$W_2$|$N_1 \times N_2$|
|Layer 2|$B_2$|$D \times 1$|
|Output|$W_{out}$|$N_2 \times O$|
|Output|$B_{out}$|$N_2 \times 1$|


In [137]:
import sklearn.datasets as data
import pandas as pd
iris = data.load_iris()

data = iris.data
targets = iris.target

In [140]:
pd.DataFrame(data).head(10)

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
5,5.4,3.9,1.7,0.4
6,4.6,3.4,1.4,0.3
7,5.0,3.4,1.5,0.2
8,4.4,2.9,1.4,0.2
9,4.9,3.1,1.5,0.1


In [143]:
data.shape

(150, 4)

In [142]:
len(np.unique(targets))

3

In [179]:
tf.reset_default_graph()

n_hidden_1 = 10
n_hidden_2 = 10
n_input = 8
n_classes = 3

weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [180]:
def multilayer_perceptron(x):
    # Hidden layer with Sigmoid activation
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with Sigmoid activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

x = tf.placeholder('float32', shape=[None, n_input])
y = tf.placeholder('float32', shape=[None, n_classes])
model = multilayer_perceptron(x)

In [189]:
from keras.utils.np_utils import to_categorical

ts = to_categorical(targets)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.5).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

training_epochs = 100

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        for i in np.arange(0, 150, 50):
            batch_x = data[i:i+50]
            batch_y = ts[i:i+50]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / 50
        # Display logs per epoch step

        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    print("Accuracy:", sess.run(accuracy, feed_dict={x: data,
                                                    y: ts}))

('Epoch:', '0001', 'cost=', '0.379013753')
('Epoch:', '0002', 'cost=', '0.080152113')
('Epoch:', '0003', 'cost=', '0.069424981')
('Epoch:', '0004', 'cost=', '0.094192142')
('Epoch:', '0005', 'cost=', '0.094033352')
('Epoch:', '0006', 'cost=', '0.082309554')
('Epoch:', '0007', 'cost=', '0.078415315')
('Epoch:', '0008', 'cost=', '0.075178365')
('Epoch:', '0009', 'cost=', '0.067710357')
('Epoch:', '0010', 'cost=', '0.064661779')
('Epoch:', '0011', 'cost=', '0.067178625')
('Epoch:', '0012', 'cost=', '0.070250067')
('Epoch:', '0013', 'cost=', '0.070917830')
('Epoch:', '0014', 'cost=', '0.070113449')
('Epoch:', '0015', 'cost=', '0.068950689')
('Epoch:', '0016', 'cost=', '0.067329752')
('Epoch:', '0017', 'cost=', '0.065801346')
('Epoch:', '0018', 'cost=', '0.065389385')
('Epoch:', '0019', 'cost=', '0.065903217')
('Epoch:', '0020', 'cost=', '0.066359922')
('Epoch:', '0021', 'cost=', '0.066312220')
('Epoch:', '0022', 'cost=', '0.065922570')
('Epoch:', '0023', 'cost=', '0.065410099')
('Epoch:', 